In [0]:
D
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}


def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select feed_id, store_id, sum(cnt) from plproxy.execute_select_nestloop($$
    select feed_id, store_id, count(app_id) as cnt
    from (select distinct feed_id, store_id, app_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and store_id not in (1) and isunique='t' ) as prod group by feed_id, store_id
        $$) tbl (feed_id smallint, store_id int, cnt bigint) group by feed_id, store_id'''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values())
        )


    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], store_id=r[1], app_id_count=r[2]) for r in rows]

result_prproxy = plproxy_row('2020-04-14', ios_feed)

In [0]:

df1 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-04-29").cache()
df1.createOrReplaceTempView("final")
df2 = spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_PREVIEW/version=2.0.0/range_type=DAY/date=2020-04-29").cache()
df2.createOrReplaceTempView("preview")

spark.sql('''SELECT DISTINCT id, store_id, category_id, platform, feed 
                FROM final 
                WHERE 
                    ( 
                     rank<=1000 AND store_id!=0 AND platform='ios'   
                     OR rank<=4000 AND store_id=0 AND platform='ios'  
                     OR rank<=1000 AND store_id!=1000 AND platform='android'  
                     OR rank<=4000 AND store_id=1000 AND platform='android' 
                    )
                AND feed IN (0,1,2,101,100,102)
                AND ( 
                        ( store_id Not In (1,2,3,4,5,6) AND platform='ios' )
                        OR
                        ( store_id Not In (1001,1002,1003,1004,1005,1006, 1007) AND platform='android' )
                    )    
                 ''').createOrReplaceTempView("final_filter")
spark.sql('''SELECT DISTINCT id, store_id, category_id, platform, feed 
                FROM preview
                WHERE 
                    ( 
                     rank<=1000 AND store_id!=0 AND platform='ios'   
                     OR rank<=4000 AND store_id=0 AND platform='ios'  
                     OR rank<=1000 AND store_id!=1000 AND platform='android'  
                     OR rank<=4000 AND store_id=1000 AND platform='android' 
                    )
                AND feed IN (0,1,2,101,100,102)
                AND ( 
                        ( store_id Not In (1,2,3,4,5,6) AND platform='ios' )
                        OR
                        ( store_id Not In (1001,1002,1003,1004,1005,1006, 1007) AND platform='android' )
                    )    
                ''').createOrReplaceTempView("preview_filter")
                
spark.sql("SELECT COUNT(1) FROM ( SELECT id FROM preview_filter EXCEPT SELECT id FROM final_filter ) AS prod").show()
spark.sql("SELECT * FROM ( SELECT id FROM preview_filter EXCEPT SELECT id FROM final_filter ) AS prod").show()

In [0]:


# spark.sql("select * from final where id=553329913").show()
# spark.sql("select * from preview where id=553329913").show()

spark.sql("select * from final_filter where id=406372707").show()
spark.sql("select * from preview_filter where id=406372707").show()


In [0]:
%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT app_id, feed_id, store_id, estimate, category_id, device_id, rank, date 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
    FROM aa.app_store_daily_estimate_0
    WHERE store_id=143619 
    and date BETWEEN '2020-11-29' AND '2020-11-29' 
    ORDER BY category_id ASC, date DESC Limit 1 \$proxy\$) 
        tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) 
ORDER BY category_id ASC, date DESC limit 1 ;

EOF

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
-- est_free_app_download + est_paid_app_download = organic_download + paid_download
-- select * from store_est_fact_v6 where date = '2016-08-26' limit 5 ;
select count(distinct app_id) from store_est_fact_v6 where date between '2020-04-29' and '2020-04-29'  limit 5 ;
-- paid_download is Zero
-- select sum(est_paid_download) from store_est_category_fact_v7 where date < '2016-08-26' ;
-- select sum(est_paid_download) from store_est_fact_v6 where date < '2016-08-26' ;
EOF

In [0]:


not_equal_data = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_15/").cache()
not_equal_data.createOrReplaceTempView("temp")
filter_df = spark.sql("select distinct * from temp where date > '2019-07-14' order by date desc, result_from").cache()
filter_df.createOrReplaceTempView("temp_filter")
diff_statisc = spark.sql('''
SELECT (p.sum_est_free_app_download - c.sum_est_free_app_download) As diff_est_free_app_download,
        (p.sum_est_paid_app_download - c.sum_est_paid_app_download) As diff_est_paid_app_download,
        (p.sum_est_revenue - c.sum_est_revenue) As diff_est_revenue,
        p.device_code, p.date
FROM temp_filter p
CROSS JOIN temp_filter c
WHERE p.result_from = 'proproxy'
AND c.result_from = 'citus'
AND p.date = c.date
AND p.device_code = c.device_code
ORDER BY date desc
''').show(20000)



# print "%table {}\t{}\t{}\t{}\t{}\t ".format( "device_code", "diff_est_free_app_download" , "diff_est_paid_app_download", "diff_est_revenue", "date")

# for x in diff_statisc:
#     print "{}\t{}\t{}\t{}\t{}\t".format(x["device_code"], x["diff_est_free_app_download"], x["diff_est_paid_app_download"], x["diff_est_revenue"], x["date"])


In [0]:

import datetime
from pyspark.sql import types as T
from pyspark.sql import functions as F

temp_date_range=["2014-11-12"]
csv_schema = T.StructType(
    [
        T.StructField("store_id", T.IntegerType(), True),
        T.StructField("date", T.DateType(), True),
        T.StructField("platform_id", T.IntegerType(), True),
        T.StructField("vertical", T.IntegerType(), True),
        T.StructField("feed", T.IntegerType(), True),
        T.StructField("id", T.LongType(), True),
        T.StructField("est", T.IntegerType(), True),
        T.StructField("category_id", T.IntegerType(), True),
        T.StructField("rank", T.IntegerType(), True)
    ]
)

df_1 = spark.read.option("basePath","s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/")\
        .schema(csv_schema)\
            .csv( "s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/{%s}/ios/sbe_est_app/*/"% (temp_date_range[0]), sep="\t")\
                .withColumn("platform", F.lit("ios"))\
                    .select('id', 'store_id', 'category_id', 'platform_id', 'vertical', 'rank', 'feed', 'est', 'date', 'platform').cache()
df_1.createOrReplaceTempView("daily_data")
spark.sql("select count(1) from daily_data where store_id = 1").show()



In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}


def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select feed_id, store_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, store_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and store_id not in (1) and isunique='t' ) as prod group by feed_id, store_id
        $$) tbl (feed_id smallint, store_id int, cnt bigint) group by feed_id, store_id'''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values())
        )


    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], store_id=r[1], metric_sum=r[2]) for r in rows]

result_prproxy = plproxy_row('2011-12-31', ios_feed)
db_id = spark.createDataFrame(result_prproxy).cache()
db_id.createOrReplaceTempView("db_result")

df_3 = spark.sql("select feed as feed_id, sum(est) as metric_sum, store_id from ( select distinct id, store_id, feed, est from daily_data where  ( ((rank<=4000 and store_id=0 ) or (rank<=1000 and store_id not in (0,1,2,3,4,5,6))) and feed in (0,1,2,101,100,102)) ) as prod group by feed, store_id").cache()
df_3.createOrReplaceTempView("unified_group")

spark.sql("select * from ( select * from db_result except all select * from unified_group ) as prod order by store_id, feed_id").show()
spark.sql("select * from (select * from unified_group except all select * from db_result) as prod order by store_id, feed_id").show()
spark.sql("select * from daily_data where store_id=143477 and feed=100 and id in (485585283, 456927907, 447065901) ").show()
spark.sql("select id, est from ( select distinct id,est from daily_data where  ( ((rank<=4000 and store_id=0 ) or (rank<=1000 and store_id not in (0,1,2,3,4,5,6))) and feed in (100) and store_id=143477)) as prod order by id desc").show(20000)

In [0]:
%%sh

# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT app_id, feed_id, store_id, estimate, category_id, device_id, rank, date 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#     FROM aa.app_store_daily_estimate_1
#     WHERE app_id In (485585283, 456927907, 447065901) 
#     AND date BETWEEN '2011-12-31' AND '2011-12-31' 
#     AND store_id = 143477 
#     AND feed_id = 100 
#     ORDER BY category_id ASC, date DESC Limit 500 \$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date ) 
# ORDER BY category_id ASC, date DESC limit 500 ;

# EOF



# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# select distinct app_id, estimate from plproxy.execute_select_nestloop(\$proxy\$ 
# select distinct app_id, estimate
#     from aa.app_store_daily_estimate_1
#     where 
#       date between '2011-12-31' and '2011-12-31' and feed_id=100 and store_id=143477  \$proxy\$) tbl 
#       (app_id BIGINT, estimate int ) order by app_id desc limit 20000 ;

# EOF



# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT date, Sum(cnt) 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT date, Count(1) as cnt from (
#         SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#         FROM aa.app_store_daily_estimate_1
#         WHERE date Between '2011-05-01' And '2011-06-01' 
#         AND rank > 1000 
#         AND store_id NOT in (0,1) ) AS prod
#         GROUP BY date \$proxy\$) tbl (date DATE, cnt BIGINT) 
#     GROUP BY date 
#     ORDER BY date;
# EOF




# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
# SELECT app_id, feed_id, store_id, estimate, category_id, device_id, rank, date 
# FROM plproxy.execute_select_nestloop(\$proxy\$ 
#     SELECT distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
#     FROM aa.app_store_daily_estimate_1
#     WHERE date BETWEEN '2011-12-31' AND '2011-12-31' 
#     AND rank > 1000 
#     AND store_id Not In (0,1) 
#     ORDER BY category_id Asc, date Desc limit 500 \$proxy\$) tbl (app_id BIGINT,feed_id SMALLINT,store_id INT, estimate INT, category_id INT, device_id SMALLINT, rank INT, date Date )
# ORDER BY category_id Asc, date Desc limit 500 ;

# EOF



PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -p 7432 -U app_bdp_usage_qa -d dailyest << EOF
SELECT date, Sum(cnt) 
FROM plproxy.execute_select_nestloop(\$proxy\$ 
    SELECT date, Count(1) as cnt from (
        SELECT Distinct app_id, feed_id, store_id, estimate, category_id, device_id, rank, date
        FROM aa.app_store_daily_estimate_1
        WHERE date In ('2014-11-12')
        AND store_id in (1) ) AS prod
        GROUP BY date \$proxy\$) tbl (date DATE, cnt BIGINT) 
    GROUP BY date 
    ORDER BY date;
EOF

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}


def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_fact_v1 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result

    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]


def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")
    
    
    if device_feed_dict.keys()[0] == 1:
        store_id = ['0','1']	
    else:	
        store_id = ['1000','1001']
    

    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({}) and isunique='t'  and (
                ( rank<=1000 AND store_id not in ({})  )
            OR ( rank<=4000 AND store_id={} )
        ))  as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values()), ','.join(store_id), str(store_id[0])
    )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], metric_sum=r[1]) for r in rows]


def generate_plploxy_result(spark, r1, r2):
    schema = StructType([
        StructField("feed_id", StringType(), True),
        StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1, schema)
    df1.createOrReplaceTempView("plploxy_ios")

    df2 = spark.createDataFrame(r2, schema)
    df2.createOrReplaceTempView("plploxy_android")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")

    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")

    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")

    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")

    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")

    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")

    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')


def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", IntegerType(), True),
    StructField("sum_est_paid_app_download", IntegerType(), True),
    StructField("sum_est_revenue", IntegerType(), True)])

    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")


def test_store_prproxy_data(spark, d):
    r1 = plproxy_row(d, ios_feed)
    r2 = plproxy_row(d, android_feed)
    citus_reseult = citus_row(d)

    d1 = generate_plploxy_result(spark,r1, r2)
    d2 = generate_citus_result(spark,citus_reseult)

    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    plproxy_except_result = spark.sql("select * from plploxy_r except all select * from citus_r")
    citus_except_result = spark.sql("select * from citus_r except all select * from plploxy_r")

    plproxy_except_result= plproxy_except_result.withColumn("result_from", F.lit("proproxy")).withColumn("date", F.lit(str(d)))
    citus_except_result = citus_except_result.withColumn("result_from", F.lit("citus")).withColumn("date", F.lit(str(d)))

    result = plproxy_except_result.union(citus_except_result)

    df_write_result = result
    # write_test_result(df_write_result)
    plproxy_except_result.show()
    citus_except_result.show()

# def write_test_result(df_write_result):
#         df_write_result.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_est_06_15/",
#                                           mode="append",
#                                           partitionBy=["date"])





start = "2014-06-27"
end = "2014-06-28"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)

for d in dates:
    print d
    test_store_prproxy_data(spark, d)


In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_category_fact_v1 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )
    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }
    settings = build_db_settings(urn, "DAILY_EST")
    if device_feed_dict.keys()[0] == 1:
        store_id = ['0','1']	
    else:	
        store_id = ['1000','1001']
    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({})  and (
                ( rank<=1000 AND store_id not in ({})  )
            OR ( rank<=4000 AND store_id={} )
        )) as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values()), ','.join(store_id), str(store_id[0])
    )
    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], metric_sum=r[1]) for r in rows]
def generate_plploxy_result(spark, r1, r2):
    schema = StructType([
        StructField("feed_id", StringType(), True),
        StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1, schema)
    df1.createOrReplaceTempView("plploxy_ios")
    df2 = spark.createDataFrame(r2, schema)
    df2.createOrReplaceTempView("plploxy_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")
    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")
    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")
    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")
    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")
    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")
    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")
    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')
    
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", IntegerType(), True),
    StructField("sum_est_paid_app_download", IntegerType(), True),
    StructField("sum_est_revenue", IntegerType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")
    
    
def test_store_prproxy_data(spark, d):
    r1 = plproxy_row(d, ios_feed)
    r2 = plproxy_row(d, android_feed)
    citus_reseult = citus_row(d)
    d1 = generate_plploxy_result(spark,r1, r2)
    d2 = generate_citus_result(spark,citus_reseult)
    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    plproxy_except_result = spark.sql("select * from plploxy_r except all select * from citus_r")
    citus_except_result = spark.sql("select * from citus_r except all select * from plploxy_r")
    plproxy_except_result= plproxy_except_result.withColumn("result_from", F.lit("proproxy")).withColumn("date", F.lit(str(d)))
    citus_except_result = citus_except_result.withColumn("result_from", F.lit("citus")).withColumn("date", F.lit(str(d)))
    result = plproxy_except_result.union(citus_except_result)
    df_write_result = result
    # write_test_result(df_write_result)
    plproxy_except_result.show()
    citus_except_result.show()
# def write_test_result(df_write_result):
#         df_write_result.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_15/",
#                                           mode="append",
#                                           partitionBy=["date"])
        # from aadatapipelinecore.core.utils.retry import retry
        # retry(write_test_result,(df_write_result,),{},interval=10)
start = "2014-06-27"
end = "2014-06-28"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)
for d in dates:
    print d
    test_store_prproxy_data(spark, d)

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F


ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}


def citus_row(date):
    def get_data_in_citus(date):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select device_code, cast(sum(est_free_app_download) as int) as est_free_app_download, cast(sum(est_paid_app_download) as int) as est_paid_app_download, cast(sum(est_revenue) as int) as est_revenue from store.store_est_category_fact_v1 where date='{}' group by device_code".format(
            date)
        db_data = query(citus_dsn_, sql)
        return db_data

    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result

    result = get_data_in_citus(date)
    return [Row(device_code=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]


def plproxy_row(date, device_feed_dict):
    urn = Urn(
        namespace='app-qa.db-check.v1',
        owner='app_qa'
    )

    def build_db_settings(urn, db_conf_name):
        template = "PG_" + db_conf_name.upper() + "_{property}"
        settings = application_settings(urn)
        host, port = getattr(settings, template.format(property='HOSTS'))[0]
        return {
            'HOST': host,
            'PORT': port,
            'NAME': getattr(settings, template.format(property='NAME')),
            'PASSWORD': getattr(settings, template.format(property='SECRET_KEY')),
            'USER': getattr(settings, template.format(property='ACCESS_ID')),
            'NODE_NUM': int(getattr(settings, template.format(property='NODE_NUM'))),
            'CLUSTER': getattr(settings, template.format(property='CLUSTER'))
        }

    settings = build_db_settings(urn, "DAILY_EST")

    sql = '''select feed_id, Cast(sum(cnt) as int) as metric_sum from plproxy.execute_select_nestloop($$
    select feed_id, sum(estimate)  as cnt
    from (select distinct app_id, feed_id, store_id, estimate,category_id, device_id from aa.app_store_daily_estimate_{}
        where date = '{}'  and  feed_id in ({})  ) as prod group by feed_id 
        $$) tbl (feed_id smallint, cnt bigint) group by feed_id '''.format(
        device_feed_dict.keys()[0], date, ','.join(device_feed_dict.values())
    )

    runner = LocalSqlRunner(settings)
    rows, _, columns = runner.select_return_columns(sql)
    return [Row(feed_id=r[0], metric_sum=r[1]) for r in rows]


def generate_plploxy_result(spark, r1, r2):
    schema = StructType([
        StructField("feed_id", StringType(), True),
        StructField("metric_sum", IntegerType(), True)])
    df1 = spark.createDataFrame(r1, schema)
    df1.createOrReplaceTempView("plploxy_ios")

    df2 = spark.createDataFrame(r2, schema)
    df2.createOrReplaceTempView("plploxy_android")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "sum_est_free_app_download"
    WHEN feed_id='100' THEN "sum_est_paid_app_download"
    WHEN feed_id='102' THEN "sum_est_revenue"
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_ios
    ''').createOrReplaceTempView("plploxy_metric")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "sum_est_free_app_download"
    WHEN feed_id='1' THEN "sum_est_paid_app_download"
    WHEN feed_id='2' THEN "sum_est_revenue"
    END AS metric from plploxy_android
    ''').createOrReplaceTempView("plploxy_metric_android")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='101' THEN "ios-tablet"
    WHEN feed_id='100' THEN "ios-tablet"
    WHEN feed_id='102' THEN "ios-tablet"
    WHEN feed_id='0' THEN "ios-phone"
    WHEN feed_id='1' THEN "ios-phone"
    WHEN feed_id='2' THEN "ios-phone"
    END AS device_code from plploxy_metric
    ''').createOrReplaceTempView("plploxy_metric_device_code")

    spark.sql('''
    SELECT *,
    CASE 
    WHEN feed_id='0' THEN "android-all"
    WHEN feed_id='1' THEN "android-all"
    WHEN feed_id='2' THEN "android-all"
    END AS device_code from plploxy_metric_android
    ''').createOrReplaceTempView("plploxy_metric_device_code_android")

    spark.sql('''
    SELECT * FROM plploxy_metric_device_code_android
    UNION ALL
    SELECT * FROM plploxy_metric_device_code
    ''').createOrReplaceTempView("all_device")

    spark.sql('''
    SELECT 
        device_code, sum_est_free_app_download , sum_est_paid_app_download , sum_est_revenue
    FROM
          all_device
     PIVOT (
        max(metric_sum) 
        FOR metric IN ('sum_est_free_app_download','sum_est_paid_app_download', 'sum_est_revenue')
      )
    ''').createOrReplaceTempView("after_pivot")

    spark.sql('''
    SELECT device_code, sum_est_free_app_download FROM after_pivot
    WHERE sum_est_free_app_download is not null
    ''').createOrReplaceTempView("after_pivot_1")

    spark.sql('''
    SELECT device_code, sum_est_paid_app_download FROM after_pivot
    WHERE sum_est_paid_app_download is not null
    ''').createOrReplaceTempView("after_pivot_2")

    spark.sql('''
    SELECT device_code, sum_est_revenue FROM after_pivot
    WHERE sum_est_revenue is not null
    ''').createOrReplaceTempView("after_pivot_3")

    return spark.sql('''
    SELECT c.device_code, c.sum_est_free_app_download, c.sum_est_paid_app_download, d.sum_est_revenue
    FROM (
    SELECT a.device_code, a.sum_est_free_app_download, b.sum_est_paid_app_download FROM after_pivot_1 a
    JOIN after_pivot_2 b
    ON a.device_code=b.device_code
    ) AS c
    JOIN after_pivot_3 d
    on c.device_code=d.device_code
    order by device_code desc
    ''')


def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("device_code", StringType(), True),
    StructField("sum_est_free_app_download", IntegerType(), True),
    StructField("sum_est_paid_app_download", IntegerType(), True),
    StructField("sum_est_revenue", IntegerType(), True)])

    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by device_code desc")


def test_store_prproxy_data(spark, d):
    r1 = plproxy_row(d, ios_feed)
    r2 = plproxy_row(d, android_feed)
    citus_reseult = citus_row(d)

    d1 = generate_plploxy_result(spark,r1, r2)
    d2 = generate_citus_result(spark,citus_reseult)

    d1.createOrReplaceTempView("plploxy_r")
    d2.createOrReplaceTempView("citus_r")
    plproxy_except_result = spark.sql("select * from plploxy_r except all select * from citus_r")
    citus_except_result = spark.sql("select * from citus_r except all select * from plploxy_r")

    plproxy_except_result= plproxy_except_result.withColumn("result_from", F.lit("proproxy")).withColumn("date", F.lit(str(d)))
    citus_except_result = citus_except_result.withColumn("result_from", F.lit("citus")).withColumn("date", F.lit(str(d)))

    result = plproxy_except_result.union(citus_except_result)

    df_write_result = result
    write_test_result(df_write_result)
    plproxy_except_result.show()
    citus_except_result.show()

def write_test_result(df_write_result):
        df_write_result.write.parquet("s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_15/",
                                          mode="append",
                                          partitionBy=["date"])

        # from aadatapipelinecore.core.utils.retry import retry
        # retry(write_test_result,(df_write_result,),{},interval=10)




start = "2020-02-04"
end = "2020-02-05"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append((str(real_date1 + datetime.timedelta(days))))
dates.sort(reverse=True)

for d in dates:
    print d
    test_store_prproxy_data(spark, d)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-11-29/ios/sbe_est_app_preview/143619/


# aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/proproxy_data_compare_category_06_15/2018-05-10



In [0]:
%%sh
